In [10]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 3.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-rr9nfv5r
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-rr9nfv5r
  Resolved https://github.com/openai/CLIP.git to commit a9b1bf5920416aaeaec965c25dd9e8f98c864f16
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369398 sha256=5e049bbe363ccbf6dea2bd579244c11168678640aad16f65331d1899e1783d36
  Stored in directory: /tmp/pip-ephem-wheel-cache-1ur99l9r/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import numpy as np
import torch
from pkg_resources import packaging
import clip
import torch.nn as nn
import torch.optim as optim

from PIL import Image
from torch.utils.data import DataLoader, Dataset


Download dataset

In [3]:
!wget https://cvg.ethz.ch/research/flash-ambient/People_Illuminations.zip

!zip -FFv People_Illuminations.zip --out downloaded_fixed.zip
!unzip downloaded_fixed.zip

--2023-05-09 00:17:51--  https://cvg.ethz.ch/research/flash-ambient/People_Illuminations.zip
Resolving cvg.ethz.ch (cvg.ethz.ch)... 129.132.85.27
Connecting to cvg.ethz.ch (cvg.ethz.ch)|129.132.85.27|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4341852743 (4.0G) [application/zip]
Saving to: ‘People_Illuminations.zip’

People_Illumination 100%[===================>]   4.04G  15.6MB/s    in 4m 31s  

2023-05-09 00:22:24 (15.3 MB/s) - ‘People_Illuminations.zip’ saved [4341852743/4341852743]

Fix archive (-FF) - salvage what can
 Found end record (EOCDR) - says expect single disk archive
Scanning for entries...
 Local ( 1      0): copying: People_Illuminations/People_001_ambient.png   (6374648 bytes)
 Local ( 1 6374721): copying: People_Illuminations/People_001_flash.png   (4617462 bytes)
 Local ( 1 10992254): copying: People_Illuminations/People_002_ambient.png   (5853938 bytes)
 Local ( 1 16846265): copying: People_Illuminations/People_002_flash.png   (4928932

Create list of image names with their respective captions

Every flash image was captioned with "flash", and every ambient image was captioned with "ambient".

In [4]:
import os

lst = [filepath for filepath in os.listdir('/content/People_Illuminations/')]

print(lst)
print(len(lst))

_, _, files = next(os.walk("/content/People_Illuminations"))
print("Number of images: ", len(files))
tot = len(files)

train_split = int(0.9 * tot)
print(train_split)

train_files = []
train_txt = []
for filepath in lst[:train_split]:
  #path = BASE_DIR + filepath
  #!mv {path} /content/dataset/training
  train_files.append('/content/People_Illuminations/' + filepath)
  if 'flash' in filepath:
    train_txt.append('flash')
  else:
    train_txt.append('ambient')

val_files = []
val_txt = []
for filepath in lst[train_split:]:
  #path = BASE_DIR + filepath
  #!mv {path} /content/dataset/training
  val_files.append('/content/People_Illuminations/' + filepath)
  if 'flash' in filepath:
    val_txt.append('flash')
  else:
    val_txt.append('ambient')

print(val_files)
print(val_txt)

['People_212_ambient.png', 'People_241_ambient.png', 'People_239_ambient.png', 'People_268_flash.png', 'People_232_flash.png', 'People_074_flash.png', 'People_091_flash.png', 'People_087_flash.png', 'People_301_ambient.png', 'People_178_flash.png', 'People_210_ambient.png', 'People_042_ambient.png', 'People_168_flash.png', 'People_192_flash.png', 'People_283_flash.png', 'People_150_flash.png', 'People_190_flash.png', 'People_133_ambient.png', 'People_217_flash.png', 'People_186_flash.png', 'People_241_flash.png', 'People_201_flash.png', 'People_182_flash.png', 'People_021_ambient.png', 'People_257_flash.png', 'People_049_flash.png', 'People_003_flash.png', 'People_115_flash.png', 'People_325_flash.png', 'People_161_ambient.png', 'People_266_flash.png', 'People_147_flash.png', 'People_207_flash.png', 'People_143_ambient.png', 'People_298_flash.png', 'People_154_flash.png', 'People_261_ambient.png', 'People_188_flash.png', 'People_084_ambient.png', 'People_256_flash.png', 'People_248_amb

In [5]:
#Referenced from : https://github.com/openai/CLIP/issues/83

# Latest Update : 18 July 2022, 09:55 GMT+7

# TO ADD :
# Gradient Checkpointing
# Filter out bias from weight decay
# Decaying learning rate with cosine schedule
# Half-precision Adam statistics
# Half-precision stochastically rounded text encoder weights were used

#BATCH_SIZE must larger than 1

BATCH_SIZE = 64
EPOCH = 3

device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(self.title)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image,title




# use your own data
list_image_path = train_files #['folder/image1.jpg','folder2/image2.jpg'] 
list_txt = train_txt #['description for image1.jpg' , 'description for image2.jpg']
dataset = image_title_dataset(list_image_path,list_txt)
train_dataloader = DataLoader(dataset,batch_size = BATCH_SIZE) #Define your own dataloader

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 


if device == "cpu":
  model.float()
else :
  clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

# add your own code to track the training progress.
for epoch in range(EPOCH):
  print("Epoch ", epoch)
  for batch in train_dataloader :
      optimizer.zero_grad()

      images,texts = batch 
    
      images= images.to(device)
      texts = texts.to(device)
    
      logits_per_image, logits_per_text = model(images, texts)

      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      total_loss.backward()
      if device == "cpu":
         optimizer.step()
      else : 
        convert_models_to_fp32(model)
        optimizer.step()
        clip.model.convert_weights(model)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 342MiB/s]


Epoch  0
Epoch  1
Epoch  2


In [8]:
torch.save({
        'epoch': epoch,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_loss,
        }, f"model_3.pt") #just change to your preferred folder/filename

In [9]:
from google.colab import files
files.download('model_3.pt') 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [12]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
checkpoint = torch.load("model_3.pt")

# Use these 3 lines if you use default model setting(not training setting) of the clip. For example, if you set context_length to 100 since your string is very long during training, then assign 100 to checkpoint['model_state_dict']["context_length"] 
checkpoint['model_state_dict']["input_resolution"] = model.input_resolution #default is 224
checkpoint['model_state_dict']["context_length"] = model.context_length # default is 77
checkpoint['model_state_dict']["vocab_size"] = model.vocab_size 

model.load_state_dict(checkpoint['model_state_dict'])

AttributeError: ignored

In [11]:
#%cp /content/model_3.pt /content/drive/MyDrive/models/imagesynth

In [18]:
checkpoint = torch.load("model_3.pt")
model.load_state_dict(checkpoint['model_state_dict'])

<All keys matched successfully>

In [19]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [20]:
torch.save(model, "new_model.pt")

In [16]:
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          